In [2]:
import sys,os
import torch
current_directory = os.getcwd()
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname(os.path.abspath(os.path.dirname(current_directory))))))
from datasets.weather_bench import WeatherDataset
from models.VariableAnalyzer.datasets.dataset import CustomDataset

In [3]:
device = ("cuda" if torch.cuda.is_available() else "cpu" )
device = torch.device(device)

weather = WeatherDataset(0, device=device)
# dataset.shape:  torch.Size([7309, 100, 1450])
original = weather.load()
dataset = CustomDataset(original, 4 * 7)

데이터셋 불러오는 중...
==== LOAD DATASET ====
 <xarray.Dataset>
Dimensions:                   (time: 7305, latitude: 28, longitude: 28,
                               level: 13)
Coordinates:
  * latitude                  (latitude) float32 39.0 38.75 38.5 ... 32.5 32.25
  * level                     (level) int64 50 100 150 200 ... 700 850 925 1000
  * longitude                 (longitude) float32 124.2 124.5 ... 130.8 131.0
  * time                      (time) datetime64[ns] 2016-12-31 ... 2021-12-31
Data variables: (12/14)
    10m_u_component_of_wind   (time, latitude, longitude) float32 ...
    10m_v_component_of_wind   (time, latitude, longitude) float32 ...
    2m_temperature            (time, latitude, longitude) float32 ...
    geopotential              (time, level, latitude, longitude) float32 ...
    mean_sea_level_pressure   (time, latitude, longitude) float32 ...
    sea_surface_temperature   (time, latitude, longitude) float32 ...
    ...                        ...
    total_cloud

Processing futures: 4it [00:03,  1.60it/s]

sea_surface_temperature


Processing futures: 14it [00:12,  1.10it/s]
Processing futures: 2it [00:00,  2.81it/s]


13.79876 sec
==== LOAD DATASET ====
 <xarray.Dataset>
Dimensions:                   (time: 7305, longitude: 14, latitude: 33,
                               level: 13)
Coordinates:
  * latitude                  (latitude) float64 21.0 22.5 24.0 ... 67.5 69.0
  * level                     (level) int64 50 100 150 200 ... 700 850 925 1000
  * longitude                 (longitude) float64 121.5 123.0 ... 139.5 141.0
  * time                      (time) datetime64[ns] 2016-12-31 ... 2021-12-31
Data variables: (12/14)
    10m_u_component_of_wind   (time, longitude, latitude) float32 ...
    10m_v_component_of_wind   (time, longitude, latitude) float32 ...
    2m_temperature            (time, longitude, latitude) float32 ...
    geopotential              (time, level, longitude, latitude) float32 ...
    mean_sea_level_pressure   (time, longitude, latitude) float32 ...
    sea_surface_temperature   (time, longitude, latitude) float32 ...
    ...                        ...
    total_cloud_cov

...

11.63829 sec
==== LOAD DATASET ====
 <xarray.Dataset>
Dimensions:                   (time: 7305, longitude: 17, latitude: 12,
                               level: 13)
Coordinates:
  * latitude                  (latitude) float64 2.903 8.71 ... 60.97 66.77
  * level                     (level) int64 50 100 150 200 ... 700 850 925 1000
  * longitude                 (longitude) float64 90.0 95.62 ... 174.4 180.0
  * time                      (time) datetime64[ns] 2016-12-31 ... 2021-12-31
Data variables: (12/14)
    10m_u_component_of_wind   (time, longitude, latitude) float32 ...
    10m_v_component_of_wind   (time, longitude, latitude) float32 ...
    2m_temperature            (time, longitude, latitude) float32 ...
    geopotential              (time, level, longitude, latitude) float32 ...
    mean_sea_level_pressure   (time, longitude, latitude) float32 ...
    sea_surface_temperature   (time, longitude, latitude) float32 ...
    ...                        ...
    total_cloud_cover 

Processing futures: 5it [00:02,  1.97it/s]

sea_surface_temperature


Processing futures: 14it [00:10,  1.35it/s]
Processing futures: 2it [00:00, 16.69it/s]


10.60235 sec
======= RESULT SHAPE =======
result_dataset.shape:  torch.Size([7305, 100, 1450])


In [5]:
print(dataset[0][0])

[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (0, 11), (0, 12), (0, 13), (0, 14), (0, 15), (0, 16), (0, 17), (0, 18), (0, 19), (0, 20), (0, 21), (0, 22), (0, 23), (0, 24), (0, 25), (0, 26), (0, 27), (0, 28), (0, 29), (0, 30), (0, 31), (0, 32), (0, 33), (0, 34), (0, 35), (0, 36), (0, 37), (0, 38), (0, 39), (0, 40), (0, 41), (0, 42), (0, 43), (0, 44), (0, 45), (0, 46), (0, 47), (0, 48), (0, 49), (0, 50), (0, 51), (0, 52), (0, 53), (0, 54), (0, 55), (0, 56), (0, 57), (0, 58), (0, 59), (0, 60), (0, 61), (0, 62), (0, 63), (0, 64), (0, 65), (0, 66), (0, 67), (0, 68), (0, 69), (0, 70), (0, 71), (0, 72), (0, 73), (0, 74), (0, 75), (0, 76), (0, 77), (0, 78), (0, 79), (0, 80), (0, 81), (0, 82), (0, 83), (0, 84), (0, 85), (0, 86), (0, 87), (0, 88), (0, 89), (0, 90), (0, 91), (0, 92), (0, 93), (0, 94), (0, 95), (0, 96), (0, 97), (0, 98), (0, 99)]
